<a href="https://www.kaggle.com/code/shaikhabdulrafay03/pima-regularization-and-weight-initialization?scriptVersionId=166580110" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')
df

In [ ]:
print("DataSet Information:")
print("-" * 30)
print('DF Shape: ' ,df.shape)
print('number of Columns: ' ,len(df.columns))
print('number of Observations: ' ,len(df))
print('Number of values in df: ' , df.count().sum())
print('Total Number of Missing values in df: ' , df.isna().sum().sum())
print('percentage of Missing values : ' ,  "{:.2f}".format(df.isna().sum().sum()/df.count().sum() *100),'%')
print('Total Number of Duplicated records in df : ' , df.duplicated().sum().sum())
print('percentage of Duplicated values : ' ,  "{:.2f}".format(df.duplicated().sum().sum()/df.count().sum() *100),'%')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df['Outcome'].value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
y = df.pop('Outcome')
x , y = ros.fit_resample(df , y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

print("Train set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)

# Hyperparameter Tuning Analysis

## Methodology 

- Hyperparameter Variations:
 - Trying different activation functions
 - Adjusting the Dropout Rates
 - Implementing L1L2 Regularization
 - Implementing GlorotNormal fro weight initializtion
 - Keeping the number of epochs constant with implemented early stopping
- I will be using StratifiedKFold for training

## Evaluation Metrics for Hyperparameter Tuning
- Training duration (number of epochs before early stopping , time in seconds)
- F1,Recall, Precision and Accuracy and Test Loss	
 


| Experiment Number | Activation Functions      | Dropout Value | Regularized (Yes/No) | Weight/Bias Initialization (Yes/No) |
|-------------------|---------------------------|---------------|----------------------|--------------------------------------|
| 1                 | sigmoid, tanh             | 0.3           | Yes                  | Yes                                  |
| 2                 | ReLU, Leaky ReLU          | 0.3           | No                   | No                                   |
| 3                 | Leaky ReLU, ELU           | 0.7           | Yes                  | Yes                                  |
| 4                 | sigmoid, ReLU             | 0.7           | No                   | No                                   |



In [ ]:
import time
from sklearn.metrics import precision_recall_fscore_support
from tensorflow import keras
from tensorflow.keras import regularizers
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import layers, models, regularizers, initializers
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LeakyReLU


In [ ]:
hidden_layers_af = ['tanh' , keras.layers.LeakyReLU(alpha=0.01) , 'elu' , 'relu']
output_layer_af = ['sigmoid' , 'relu' , keras.layers.LeakyReLU(alpha=0.01) , 'sigmoid']
dropouts = [0.3 , 0.3 , 0.7 , 0.7]
regularization = [True , True ,False , False]
weight_init = [True , False ,True , False]

f1 = []
pre = []
re = []

times = []
eps = []
train_acc = []
test_loss = []
test_accu = []

historys = []
modelss = []

In [ ]:
for hl , ol , dr , r , wi in zip(hidden_layers_af , output_layer_af , dropouts , regularization , weight_init):


    if r == True :
        reg = regularizers.l1_l2(l1=1e-5, l2=1e-4)
    else: 
        reg = None
    
    if wi == True :
        w_init = initializers.GlorotNormal()
        b_init = initializers.Zeros()
    else : 
        w_init = None
        b_init = None
        
    model = models.Sequential([
        layers.BatchNormalization(input_shape=[X_train.shape[1]]),
        layers.Dense(256, activation= hl , kernel_regularizer= reg,
                     kernel_initializer=w_init , bias_initializer=b_init),
        layers.BatchNormalization(),
        layers.Dropout(dr),
        layers.Dense(256, activation=hl , kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                     kernel_initializer=w_init, bias_initializer=b_init),
        layers.BatchNormalization(),
        layers.Dropout(dr),
        layers.Dense(512, activation=hl , kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                     kernel_initializer=w_init, bias_initializer=b_init),
        layers.BatchNormalization(),
        layers.Dropout(dr),
        layers.Dense(1, activation=ol),
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    early_stopping = keras.callbacks.EarlyStopping(
        patience=5,
        min_delta=0.001,
        restore_best_weights=True,
    )


    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    t = []
    e = []
    ta = []
    for train_indices, val_indices in k_fold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_train_fold, y_val_fold = y_train.iloc[train_indices], y_train.iloc[val_indices]

        start_time = time.time()
        history = model.fit(
            X_train_fold, y_train_fold,
            validation_data=(X_val_fold, y_val_fold),
            batch_size=256,
            epochs=200,
            callbacks=[early_stopping],
        )
        end_time = time.time()
        training_time = end_time - start_time
        t.append(training_time)
        e.append(len(history.history['loss']))
        ta.append(history.history['accuracy'][-1])
    
    times.append(sum(t) / float(len(t)))
    eps.append(sum(e) / float(len(e)))
    train_acc.append(sum(ta) / float(len(ta)))

    eval = model.evaluate(X_test , y_test)
    print('Test Loss and Accuracy : ' , eval)
    test_loss.append(eval[0])
    test_accu.append(eval[1])
    
    predictions = model.predict(X_test)
    prediction_labels = []

    for prediction in predictions:
      prediction_labels.append(np.argmax(prediction))
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, prediction_labels, average='binary')
    f1.append(f1_score)
    pre.append(precision)
    re.append(recall)
    modelss.append(model)
    historys.append(history)

# Desicion Boundaries

In [ ]:
def db(model, ax , i):
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)

    # Apply PCA to reduce dimensions to 2
    pca = PCA(n_components=2)
    train_images_pca = pca.fit_transform(X_train_std)

    # Define meshgrid for decision boundary
    x_min, x_max = train_images_pca[:, 0].min() - 0.5, train_images_pca[:, 0].max() + 0.5
    y_min, y_max = train_images_pca[:, 1].min() - 0.5, train_images_pca[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

    meshgrid_points = np.c_[xx.ravel(), yy.ravel()]
    meshgrid_points_original = pca.inverse_transform(meshgrid_points)

    # Predict on meshgrid points
    predictions = model.predict(meshgrid_points_original)
    Z = np.argmax(predictions, axis=1)
    Z = Z.reshape(xx.shape)

    # Plot decision boundary
    ax.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_title(f'Model {i}')


fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))


db(modelss[0], axes[0, 0] , 1)
db(modelss[1], axes[0, 1] , 2)
db(modelss[2] , axes[1, 0] ,3)
db(modelss[3] , axes[1, 1] ,4)

plt.tight_layout()
plt.show()

# Results

In [ ]:
output_layer_af = ['sigmoid' , 'relu' , "LeakyRelu" , 'sigmoid']
hidden_layers_af = ['tanh' , 'LeakyRelu', 'elu' , 'relu']

d = {
'Hidden Layer AF' : hidden_layers_af,
'Output Layer AF' : output_layer_af,
'DropOut Rate' : dropouts,
'Regulariztion' : regularization,
'Weights Initialzation' : weight_init,
'Train time' : times,
'Epochs' : eps,
'Train Accuracy' : train_acc,
'Test Loss' : test_loss,
'Test Accuracy' : test_accu,
'F1 Score' : f1,
'Precsion' : pre,
'Recall' : re
}

dd = pd.DataFrame(d)
dd



# Results:
#### Experiment 1:
- Shows underfitting
- regulariztion and initializatuion are preventing overfitting

#### Experiment 2:
- Shows overfitting
- No initializatuion are causing overfitting

#### Experiment 3:
- Shows underfitting
- high dropout rate causing underfitting 

#### Experiment 1:
- Shows underfitting
- high dropout rate causing underfitting 
 


In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, regularizers, initializers
from scipy.stats import uniform, randint
import numpy as np

# Assuming X_train, y_train, X_test, y_test are your data
# Adjust the hyperparameter lists based on your requirements
hidden_layers_af = ['tanh', keras.layers.LeakyReLU(alpha=0.01), 'elu', 'relu']
output_layer_af = ['sigmoid', 'relu', keras.layers.LeakyReLU(alpha=0.01), 'sigmoid']
dropouts = [0.3, 0.3, 0.7, 0.7]
regularization = [True, False]
weight_init = [True, False]
batch_sizes = [32, 64, 128, 256]
learning_rates = uniform(loc=0.0001, scale=0.1)

# Define the number of folds for cross-validation
n_splits = 5
k_fold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a dictionary of hyperparameters and their possible values
param_dist = {
    'hl': hidden_layers_af,
    'ol': output_layer_af,
    'dr': dropouts,
    'reg': regularization,
    'w_init': weight_init,
    'batch_size': batch_sizes,
    'learning_rate': learning_rates
}


random_search_results = []


n_iter = 10
for _ in range(n_iter):
   
    hyperparams = {key: np.random.choice(values) if key != 'learning_rate' else values.rvs() for key, values in param_dist.items()}

    accuracies = []

    for train_indices, val_indices in k_fold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_train_fold, y_val_fold = y_train.iloc[train_indices], y_train.iloc[val_indices]

        model = Sequential([
            layers.BatchNormalization(input_shape=[X_train.shape[1]]),
            layers.Dense(256, activation=hyperparams['hl'], kernel_regularizer=(regularizers.l1_l2(l1=1e-5, l2=1e-4) if hyperparams['reg'] else None),
                         kernel_initializer=(initializers.GlorotNormal() if hyperparams['w_init'] else initializers.Zeros()),
                         bias_initializer=initializers.Zeros()),
            layers.BatchNormalization(),
            layers.Dropout(hyperparams['dr']),
            layers.Dense(256, activation=hyperparams['hl'], kernel_regularizer=(regularizers.l1_l2(l1=1e-5, l2=1e-4) if hyperparams['reg'] else None),
                         kernel_initializer=(initializers.GlorotNormal() if hyperparams['w_init'] else initializers.Zeros()),
                         bias_initializer=initializers.Zeros()),
            layers.BatchNormalization(),
            layers.Dropout(hyperparams['dr']),
            layers.Dense(512, activation=hyperparams['hl'], kernel_regularizer=(regularizers.l1_l2(l1=1e-5, l2=1e-4) if hyperparams['reg'] else None),
                         kernel_initializer=(initializers.GlorotNormal() if hyperparams['w_init'] else initializers.Zeros()),
                         bias_initializer=initializers.Zeros()),
            layers.BatchNormalization(),
            layers.Dropout(hyperparams['dr']),
            layers.Dense(1, activation=hyperparams['ol']),
        ])

        model.compile(
            optimizer=Adam(learning_rate=hyperparams['learning_rate']),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        history = model.fit(
            X_train_fold, y_train_fold,
            validation_data=(X_val_fold, y_val_fold),
            batch_size=hyperparams['batch_size'],
            epochs=200,
            callbacks=[early_stopping],
            verbose=0,
        )

        y_pred_val = (model.predict(X_val_fold) > 0.5).astype(int)
        accuracy_val = accuracy_score(y_val_fold, y_pred_val)
        accuracies.append(accuracy_val)

    avg_accuracy = sum(accuracies) / len(accuracies)

    random_search_results.append((hyperparams['hl'], hyperparams['ol'], hyperparams['dr'], hyperparams['reg'],
                                  hyperparams['w_init'], hyperparams['batch_size'], hyperparams['learning_rate'], avg_accuracy))

best_hyperparams = max(random_search_results, key=lambda x: x[-1])

print("Best Hyperparameters:")
print("Hidden Layer Activation:", best_hyperparams[0])
print("Output Layer Activation:", best_hyperparams[1])
print("Dropout Rate:", best_hyperparams[2])
print("Regularization:", best_hyperparams[3])
print("Weight Initialization:", best_hyperparams[4])
print("Batch Size:", best_hyperparams[5])
print("Learning Rate:", best_hyperparams[6])
print("Average Accuracy:", best_hyperparams[7])


In [ ]:
    
for train_indices, val_indices in k_fold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_train_fold, y_val_fold = y_train.iloc[train_indices], y_train.iloc[val_indices]

        model = Sequential([
            layers.BatchNormalization(input_shape=[X_train.shape[1]]),
            layers.Dense(256, activation= keras.layers.LeakyReLU(alpha=0.01) , kernel_regularizer= None,
                         kernel_initializer= initializers.Zeros(),
                         bias_initializer=initializers.Zeros()),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
             layers.Dense(256, activation= keras.layers.LeakyReLU(alpha=0.01) , kernel_regularizer= None,
                         kernel_initializer= initializers.Zeros(),
                         bias_initializer=initializers.Zeros()),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
             layers.Dense(512, activation= keras.layers.LeakyReLU(alpha=0.01) , kernel_regularizer= None,
                         kernel_initializer= initializers.Zeros(),
                         bias_initializer=initializers.Zeros()),
            layers.BatchNormalization(),
            layers.Dropout(0.3),            
            layers.BatchNormalization(),
            layers.Dense(1, activation=hyperparams['ol']),
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.003518474982689479),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        history = model.fit(
            X_train_fold, y_train_fold,
            validation_data=(X_val_fold, y_val_fold),
            batch_size=256,
            epochs=200,
            callbacks=[early_stopping],
            verbose=0,
        )
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="categorical_crossentropy")
history_df.loc[:, ['accuracy', 'val_accuracy']].plot(title="Accuracy")

In [ ]:

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

pca = PCA(n_components=2)
train_images_pca = pca.fit_transform(X_train_std)

x_min, x_max = train_images_pca[:, 0].min() - 0.5, train_images_pca[:, 0].max() + 0.5
y_min, y_max = train_images_pca[:, 1].min() - 0.5, train_images_pca[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

meshgrid_points = np.c_[xx.ravel(), yy.ravel()]
meshgrid_points_original = pca.inverse_transform(meshgrid_points)

predictions = model.predict(meshgrid_points_original)
Z = np.argmax(predictions, axis=1)
Z = Z.reshape(xx.shape)


plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Decision Boundary')
plt.show()
